In [1]:
import pandas as pd
from pyvi import ViUtils
import re

In [2]:
dmx_df = pd.read_csv('../data/dienmayxanh.csv', encoding='utf-8')
for column in dmx_df:
    print(column)

TÊN SẢN PHẨM
GIÁ
CHIP
KÍCH THƯỚC MÀN HINH
CÔNG NGHỆ MÀN HÌNH
PIN
SIM
RAM
ROM


In [3]:
dmx_df.head()

,TÊN SẢN PHẨM,GIÁ,CHIP,KÍCH THƯỚC MÀN HINH,CÔNG NGHỆ MÀN HÌNH,PIN,SIM,RAM,ROM
0,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,27.990.000₫,Snapdragon 8 Gen 1 8 nhân,"6.8"" - Tần số quét 120 Hz",Dynamic AMOLED 2X,5000 mAh,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,8 GB,128 GB
1,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,27.990.000₫,Snapdragon 8 Gen 1 8 nhân,"6.8"" - Tần số quét 120 Hz",Dynamic AMOLED 2X,5000 mAh,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,8 GB,128 GB
2,Điện thoại iPhone 11 64GB,NaN,Apple A13 Bionic 6 nhân,"6.1"" - Tần số quét 60 Hz",IPS LCD,3110 mAh,1 Nano SIM & 1 eSIM,4 GB,64 GB
3,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,27.990.000₫,Snapdragon 8 Gen 1 8 nhân,"6.8"" - Tần số quét 120 Hz",Dynamic AMOLED 2X,5000 mAh,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,8 GB,128 GB
4,Điện thoại iPhone 11 64GB,NaN,Apple A13 Bionic 6 nhân,"6.1"" - Tần số quét 60 Hz",IPS LCD,3110 mAh,1 Nano SIM & 1 eSIM,4 GB,64 GB


In [4]:
print(len(dmx_df))

101


# Xử lý tên cột

In [5]:
dmx_df = dmx_df.rename(columns={"TÊN SẢN PHẨM": "Tên"})

for column in dmx_df.columns:
    dmx_df = dmx_df.rename(columns={column: ViUtils.remove_accents(column).decode("utf-8").lower().strip().replace(' ', '_')})

# Xử lý trường tên

In [6]:
dmx_df = dmx_df[dmx_df['ten'].notna()]
dmx_df['ten'] = dmx_df['ten'].apply(lambda s: str(s).replace('|', '').replace('Chính hãng', '').replace('VN/A', '').replace('I', '').replace('Điện thoại', '').strip())
dmx_df['ten'] = dmx_df['ten'].apply(lambda s: ViUtils.remove_accents(s).decode("utf-8"))

# Xử lý trường giá

In [7]:
dmx_df = dmx_df[dmx_df['gia'].notna()]
dmx_df['gia'] = dmx_df['gia'].apply(lambda s: str(s).replace('₫', '').replace('.', '').replace('*', '').strip())
dmx_df['gia'] = dmx_df['gia'].apply(lambda s: int(s))

# Xử lý trường kích thước màn hình

In [8]:
def process_screen_size(s):
    try:
        size = re.findall(r"[-+]?(?:\d*\.\d+|\d+)", s)
        res = 0
        for si in size:
            si = float(si)
            if si < 20:
                res += si
        return res
    except:
        return 0

dmx_df['kich_thuoc_man_hinh'] = dmx_df['kich_thuoc_man_hinh'].apply(process_screen_size)

# Xử lý trường bộ nhớ trong

In [9]:
def process_rom(s):
    try:
        if 'GB' in s:
            return float(s[:-3])
        elif 'MB' in s:
            return float(s[:-3])/1024
        else:
            return float(s[:-3])*1024
    except:
        return 0

dmx_df['rom'] = dmx_df['rom'].apply(process_rom)

# Xử lý trường RAM

In [10]:
def process_ram(s):
    try:
        if 'GB' in s:
            return float(s[:-3])
        elif 'MB' in s:
            return float(s[:-3])/1024
        else:
            return float(s[:-3])*1024
    except:
        return 0

dmx_df['ram'] = dmx_df['ram'].apply(process_ram)

In [11]:
for column in dmx_df.columns:
    print(column)

ten
gia
chip
kich_thuoc_man_hinh
cong_nghe_man_hinh
pin
sim
ram
rom


In [12]:
dmx_df = dmx_df.drop_duplicates()

In [13]:
dmx_df.to_csv('../clean_data/dienmayxanh.csv', index=False)